# Cartpole Game

Deep-Q-Learning with Keras and Gym, based on [Keon Kim's](https://github.com/keon/deep-q-learning) code.

In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

#### Import dependencies

In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


#### Set hyperparameters

In [3]:
n_episodes = 1000 # default 1000

#### Define class for Deep-Q-Learning agent

In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

#### Set other parameters (some of these should be moved to top of file)

In [5]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

In [6]:
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(5000):
        env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, n_episodes, time, agent.epsilon))
            break
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
#     if e % 10 == 0:
#         agent.save("./save/cartpole-dqn.h5")

episode: 0/1000, score: 8, e: 1.0
episode: 1/1000, score: 28, e: 1.0
episode: 2/1000, score: 19, e: 0.99
episode: 3/1000, score: 12, e: 0.99
episode: 4/1000, score: 13, e: 0.99
episode: 5/1000, score: 33, e: 0.98
episode: 6/1000, score: 26, e: 0.98
episode: 7/1000, score: 13, e: 0.97
episode: 8/1000, score: 42, e: 0.97
episode: 9/1000, score: 10, e: 0.96
episode: 10/1000, score: 13, e: 0.96
episode: 11/1000, score: 10, e: 0.95
episode: 12/1000, score: 40, e: 0.95
episode: 13/1000, score: 27, e: 0.94
episode: 14/1000, score: 12, e: 0.94
episode: 15/1000, score: 9, e: 0.93
episode: 16/1000, score: 38, e: 0.93
episode: 17/1000, score: 17, e: 0.92
episode: 18/1000, score: 11, e: 0.92
episode: 19/1000, score: 21, e: 0.91
episode: 20/1000, score: 11, e: 0.91
episode: 21/1000, score: 36, e: 0.9
episode: 22/1000, score: 38, e: 0.9
episode: 23/1000, score: 31, e: 0.9
episode: 24/1000, score: 15, e: 0.89
episode: 25/1000, score: 17, e: 0.89
episode: 26/1000, score: 18, e: 0.88
episode: 27/1000, 

KeyboardInterrupt: 